In [17]:
# Load packages we need
import sys
import os
import time

import numpy as np
import pandas as pd
import sklearn

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.datasets import fashion_mnist, cifar10, imdb

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


# import layers and callbacks we may use (may not be a complete list)
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model, Sequential

from tensorflow.python.keras.utils import layer_utils


from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 14})

# Load the TensorBoard notebook extension
#%load_ext tensorboard


# Let's check our software versions
print('------------')
print('### Python version: ' + __import__('sys').version)
print('### NumPy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('### Tensorflow version: ' + tf.__version__)
print('------------')

def var_exists(var_name):
    return (var_name in globals() or var_name in locals())
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array


------------
### Python version: 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]
### NumPy version: 1.26.4
### Scikit-learn version: 1.4.1.post1
### Tensorflow version: 2.16.1
------------


In [2]:
# Load the csv

df = pd.read_csv("../data/train.csv")

# Create a list with all the images names
img_path_list = os.listdir("../data/train_images")

# Include a column in the dataframe with the path to the image
imgs_path = ("../data/train_images/")
for img in img_path_list:
    df["img_path"] = imgs_path + img
    

print(df["img_path"][0])
    
def load_image(img_path):
    img = Image.open(img_path).convert("RGB").resize((224, 224))
    img_array = np.array(img)  # Convert PIL Image to numpy array
    return img_array

# Apply the function to each row in the DataFrame to load the images
df["img_rgb"] = df["img_path"].apply(lambda x: load_image(x))
    
print(df.head())

../data/train_images/3150477025.jpg
         image_id  label                             img_path  \
0  1000015157.jpg      0  ../data/train_images/3150477025.jpg   
1  1000201771.jpg      3  ../data/train_images/3150477025.jpg   
2   100042118.jpg      1  ../data/train_images/3150477025.jpg   
3  1000723321.jpg      1  ../data/train_images/3150477025.jpg   
4  1000812911.jpg      3  ../data/train_images/3150477025.jpg   

                                             img_rgb  
0  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
1  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
2  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
3  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
4  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  


In [10]:
all_x = np.stack(df["img_rgb"].values)


print(type(all_x), all_x.shape)

all_y = np.array(df["label"])

print(type(all_y), all_y.shape)

<class 'numpy.ndarray'> (21397, 224, 224, 3)
<class 'numpy.ndarray'> (21397,)


In [11]:
all_x[0].shape

(224, 224, 3)

In [29]:
train_x, aux_x, train_y, aux_y = train_test_split(all_x, all_y, test_size=0.2, random_state=32)
test_x, val_x, test_y, val_y = train_test_split(aux_x, aux_y, test_size=0.5, random_state=32)



In [31]:
def create_compile_cnn(input_shape=(224, 224, 3), num_outputs=5, verbose=False):
    

    # Create a Sequential model
    model = Sequential()

    # Add a convolutional layer
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

    # Add a max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add another convolutional layer
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

    # Add another max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output of the convolutional layers
    model.add(Flatten())

    # Add a dense (fully connected) layer
    model.add(Dense(128, activation='relu'))

    # Add the output layer
    model.add(Dense(num_classes, activation='softmax'))  # Assuming num_classes is defined

    
    if verbose:
        model.summary()
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [32]:
model = create_compile_cnn(verbose=True)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,037 (91.20 MB)

 Trainable params: 23,908,037 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
 # Convert labels to one-hot encoding
train_y_one_hot = to_categorical(train_y)
val_y_one_hot = to_categorical(val_y)
test_y_one_hot = to_categorical(test_y)

# Train the model
model.fit(train_x, train_y_one_hot, batch_size=32, epochs=10, validation_data=(val_x, val_y_one_hot))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_x, test_y_one_hot)
print('Test accuracy:', test_acc)

Epoch 1/10
 71/535 ━━━━━━━━━━━━━━━━━━━━ 2:51 369ms/step - accuracy: 0.4871 - loss: 559.8507

KeyboardInterrupt: 

In [ ]:
print("train_x shape:", train_x.shape, "data type:", train_x.dtype)
print("train_y shape:", train_y.shape, "data type:", train_y.dtype)